This the notebook version of the example in read the doc.
The following lines present a small example of simulation the sap flux from an Arabidopsis de-topped root plunged in a hydroponic solution at a hydrostatic pressure of 0.4 Mpa when its  base is at the atmospheric pressure.

In [13]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['../src'])

Python 3.8.12 | packaged by conda-forge | (default, Jan 30 2022, 23:42:07) 
[GCC 9.4.0] on linux


In [14]:
from hydroroot.display import plot
from hydroroot.read_file import read_archi_data
from hydroroot.main import hydroroot_flow, root_builder

Read the architecture file

In [15]:
df = read_archi_data('data/plant-01.txt')

Building the MTG from the file, and some properties needed for the calculation

In [4]:
g, primary_length, total_length, surface, seed = root_builder(df=df, segment_length=1.0e-4)

Some conductance data versus distance to tip

In [5]:
k_radial_data=([0, 0.2],[30.0,30.0])
K_axial_data=([0, 0.2],[3.0e-7,4.0e-4])

Flux and equivalent conductance calculation, for a root in an external hydroponic medium at 0.4 MPa, its base at 0.1 MPa, and with the conductances set above.

In [6]:
g, keq, jv = hydroroot_flow(g, psi_e = 0.4, psi_base = 0.1, axial_conductivity_data = K_axial_data, radial_conductivity_data = k_radial_data)

In [7]:
print('equivalent root conductance (microL/s/MPa): ',keq, 'sap flux (microL/s): ', jv)

equivalent root conductance (microL/s/MPa):  0.009596381061843703 sap flux (microL/s):  0.002878914318553111


Display the local water uptake heatmap in 3D

In [9]:
%gui qt
plot(g, prop_cmap='j') # j is the radial flux in ul/s

You may change the property to display to the hydrostatic pressure inside the xylem vessels for instance

In [10]:
plot(g, prop_cmap='psi_in')

You may change the radial conductivity and see the impact on the water uptake

In [11]:
k_radial_data=([0, 0.2],[300.0,300.0])
g, keq, jv = hydroroot_flow(g, psi_e = 0.4, psi_base = 0.1, axial_conductivity_data = K_axial_data, radial_conductivity_data = k_radial_data)
print('sap flux (microL/s): ', jv)
plot(g, prop_cmap='j')

sap flux (microL/s):  0.010609107156851389


Or the axial conductance

In [12]:
k_radial_data=([0, 0.2],[30.0,30.0])
K_axial_data=([0, 0.2],[3.0e-7,1.0e-4])
g, keq, jv = hydroroot_flow(g, psi_e = 0.4, psi_base = 0.1, axial_conductivity_data = K_axial_data, radial_conductivity_data = k_radial_data)
print('sap flux (microL/s): ', jv)
plot(g, prop_cmap='j')

sap flux (microL/s):  0.0019166890917623359
